The aim of this assignment is coding the Newton's method for multi-dimensional functions without including any additional packages. The minimum values of 2 following objective functions will be found:

$$f(x_1,x_2) = (5x_1-x_2)^2 + (x_1-2)^2 + x_1 - 2x_2 + 12$$

$$f(x_1,x_2) = 100(x_2-x_1^2)^2 + (1-x_1)^2$$

In [4]:
objective1 <- expression(
  (5*x1 - x2)^4 + (x1 - 2)^2 + x1 -2*x2 + 12
)

objective2 <- expression(
  100*(x2-x1^2)^2 + (1-x1)^2
)

Here is the function for calculating the norm of a vector.

In [5]:
norm_vec <- function(vec){
  sqrt(sum(vec^2))
}

Get Hessian function computes the Hessian matrix of a given function.

In [6]:
get_hessian <- function(exprs,xk){
  xk1 <- xk[1]
  xk2 <- xk[2]
  h1 <- eval(D(D(exprs,"x1"),"x1"),envir=list(x1=xk1,x2=xk2))
  h2 <- eval(D(D(exprs,"x1"),"x2"),envir=list(x1=xk1,x2=xk2))
  h3 <- eval(D(D(exprs,"x2"),"x1"),envir=list(x1=xk1,x2=xk2))
  h4 <- eval(D(D(exprs,"x2"),"x2"),envir=list(x1=xk1,x2=xk2))
  hes <- matrix(c(h1,h3,h2,h4),nrow=2,ncol=2)
  return(hes)
}

Golden Section Method function is used for finding the minimum of a one-dimensional function.

In [7]:
GoldenSectionMethod <- function(a,b,e2,func) {
  gamma <- 1.618
  x <- b - (1/gamma)*(b-a)
  y <- a + (1/gamma)*(b-a)
  fx <- func(x)
  fy <- func(y)
  
  while (b-a>=e2) {
    if(fx > fy) {
      a <- x
      x <- y
      y <- a + (1/gamma)*(b-a)
      fx <- fy
      fy <- func(y)
    }
    
    else {
      b <- y
      y <- x
      x <- b - (1/gamma)*(b-a)
      fy <- fx
      fx <- func(x)
    }
  }
  
  return(x)
}

Argmin function is used to find argument minimum of the alpha parameter.

In [8]:
argmin <- function(objective,xk,e2,direction){
  
  func <- function(alpha){
    return(objective(xk+(alpha*direction)))
  }
  min <- GoldenSectionMethod(-100,100,e2,func)
  return(min)
}

Newton's method for multi-dimensional functions is shown below.

In [9]:
newtons_method <- function(e1,e2,exprs,x0){
  
  objective_st <- function(xvec){
    x1_st <- xvec[1]
    x2_st <- xvec[2]
    return(eval(exprs,envir = list(x1=x1_st,x2=x2_st)))
  }
  k <- 0
  x <- list()
  alpha <- list()
  direction_list <- list()
  x[[as.character(k)]] <- x0
  
  while(TRUE){
    hes <- get_hessian(exprs,x[[as.character(k)]])
    g1  <- eval(D(exprs,"x1"),envir = list(x1=x[[as.character(k)]][1],x2=x[[as.character(k)]][2]))
    g2  <- eval(D(exprs,"x2"),envir = list(x1=x[[as.character(k)]][1],x2=x[[as.character(k)]][2]))
    grad <- c(-g1,-g2)
    direction <- solve(hes) %*% grad
    direction_list[[as.character(k)]] <- direction
    alpha[[as.character(k)]] <- argmin(objective_st,x[[as.character(k)]],e2,direction)
    x[[as.character(k+1)]] <- x[[as.character(k)]]+(alpha[[as.character(k)]]*direction)
    k <- k + 1
    if(abs(objective_st(x[[as.character(k)]])-objective_st(x[[as.character(k-1)]]))<e1){
      return(list("x"=x,"alpha"=alpha,"direction"=direction_list))
    }
  }
}

Give result function is used for compute the results of the Newton's method with different parameters.

In [10]:
give_result <- function(e1,e2,exprs,x0){
  
  objective_res <- function(xvec){
    x1_res <- xvec[1]
    x2_res <- xvec[2]
    eval(exprs,envir = list(x1=x1_res,x2=x2_res))
  }
  
  sol <- newtons_method(e1,e2,exprs,x0)
  x1 <- numeric(0)
  x2 <- numeric(0)
  f <- numeric(0)
  a <- numeric(0)
  d1 <- numeric(0)
  d2 <- numeric(0)
  
  for(i in 0:length(sol$x)){
    x1 <- c(x1,sol$x[[as.character(i)]][1])
    x2 <- c(x2,sol$x[[as.character(i)]][2])
    a <- c(a,sol$alpha[[as.character(i)]])
    f <- c(f,objective_res(sol$x[[as.character(i)]]))
    d1 <- c(d1,sol$direction[[as.character(i)]][1])
    d2 <- c(d2,sol$direction[[as.character(i)]][2])
  }
  
  x1 <- as.character(round(x1,3))
  x2 <- as.character(round(x2,3))
  d1 <- as.character(round(d1,3))
  d2 <- as.character(round(d2,3))
  f <- round(f,3)
  X <- paste(x1,x2,sep=" , ")
  D <- paste(d1,d2,sep=" , ")
  Xk1  <- c(X[2:(length(X))],NA)
  a[length(a)+1]=NA
  D[length(D)+1]=NA
  
  results <- data.frame("Iteration"=(0:(length(sol$x)-1)),"X(k)"=X,"f(x1,x2)"=f,"d"=D,"alpha"=a,"X(k+1)"=Xk1)
  return(results)
}

Here is the result table of objective function 1, with different epsilon values and different initial points.

In [11]:
give_result(0.001,0.005,objective1,c(31,20))
give_result(0.005,0.005,objective1,c(100,100))

Iteration,X.k.,f.x1.x2.,d,alpha,X.k.1.
<int>,<chr>,<dbl>,<chr>,<dbl>,<chr>
0,"31 , 20",332151469.000,"-24.5 , -77.5",2.9487185,"-41.244 , -208.526"
1,"-41.244 , -208.526",2286.175,"47.744 , 239.519",1.0029679,"6.642 , 31.704"
2,"6.642 , 31.704",-18.093,"-0.142 , -0.133",3.9464521,"6.082 , 31.178"
3,"6.082 , 31.178",-27.263,"0.418 , 2.117",0.9991454,"6.5 , 33.293"
4,"6.5 , 33.293",-27.441,"0 , 0.001",1.0008245,"6.5 , 33.294"
5,"6.5 , 33.294",-27.441,NA,NA,NA


Iteration,X.k.,f.x1.x2.,d,alpha,X.k.1.
<int>,<chr>,<dbl>,<chr>,<dbl>,<chr>
0,"100 , 100",2.560001e+10,"-93.5 , -334.167",2.9682689,"-177.533 , -891.897"
1,"-177.533 , -891.897",3.417081e+04,"184.033 , 921.585",1.0008245,"6.652 , 30.449"
2,"6.652 , 30.449",4.174700e+01,"-0.152 , 0.199",3.7434357,"6.084 , 31.194"
3,"6.084 , 31.194",-2.726600e+01,"0.416 , 2.1",0.9991454,"6.5 , 33.292"
4,"6.5 , 33.292",-2.744100e+01,"0 , 0.001",1.0010567,"6.5 , 33.294"
5,"6.5 , 33.294",-2.744100e+01,NA,NA,NA


Here is the result table of objective function 2, with different epsilon values and different initial points.

In [12]:
give_result(0.001,0.005,objective2,c(5,5))
give_result(0.005,0.005,objective2,c(100,100))

Iteration,X.k.,f.x1.x2.,d,alpha,X.k.1.
<int>,<chr>,<dbl>,<chr>,<dbl>,<chr>
0,"5 , 5",40016.000,"-0.001 , 19.99",0.9991454,"4.999 , 24.973"
1,"4.999 , 24.973",16.021,"-0.905 , -9.031",0.2793476,"4.746 , 22.45"
2,"4.746 , 22.45",14.615,"-0.231 , -2.113",1.5613663,"4.386 , 19.152"
3,"4.386 , 19.152",12.220,"-0.184 , -1.531",2.3773330,"3.948 , 15.513"
4,"3.948 , 15.513",9.216,"-0.19 , -1.428",1.9710821,"3.573 , 12.697"
5,"3.573 , 12.697",7.110,"-0.172 , -1.157",2.2352806,"3.189 , 10.112"
6,"3.189 , 10.112",5.163,"-0.166 , -1.001",2.0995224,"2.84 , 8.011"
7,"2.84 , 8.011",3.691,"-0.153 , -0.813",2.2111070,"2.502 , 6.213"
8,"2.502 , 6.213",2.482,"-0.143 , -0.668",2.1754283,"2.191 , 4.759"


Iteration,X.k.,f.x1.x2.,d,alpha,X.k.1.
<int>,<chr>,<dbl>,<chr>,<dbl>,<chr>
0,"100 , 100",9.801010e+09,"0 , 9899.99",0.999145357,"100 , 9991.529"
1,"100 , 9991.529",1.695979e+04,"-0.058 , -3.233",1.001056720,"99.941 , 9988.293"
2,"99.941 , 9988.293",9.789407e+03,"960.629 , 192013.174",-0.001491944,"98.508 , 9701.82"
3,"98.508 , 9701.82",9.927508e+03,"-0.237 , -44.726",1.090205132,"98.249 , 9653.06"
4,"98.249 , 9653.06",9.458831e+03,"4.311 , 846.952",-0.196709287,"97.401 , 9486.456"
5,"97.401 , 9486.456",9.326651e+03,"-0.827 , -160.452",1.137573837,"96.461 , 9303.93"
6,"96.461 , 9303.93",9.177573e+03,"-0.589 , -112.915",2.233233777,"95.145 , 9051.765"
7,"95.145 , 9051.765",8.918046e+03,"-0.631 , -119.416",1.889074071,"93.952 , 8826.179"
8,"93.952 , 8826.179",8.698491e+03,"-0.604 , -112.719",2.068759974,"92.702 , 8592.99"
